In [ ]:
import keras
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class_type = ['COVID','non-COVID']
data_dir = '/content/drive/MyDrive/Colab Notebooks/data_covid/data'
train_dir = os.path.join(data_dir)

In [ ]:
train_data = []
for defects_id, sp in enumerate(class_type):
    print(defects_id, sp)
    for file in os.listdir(os.path.join(train_dir, sp)):
      train_data.append(['{}/{}'.format(sp,file),defects_id, sp])

print(train_data)
train = pd. DataFrame(train_data, columns = ['File', 'DiseaseID', 'Disease Type'])
print(train)
SEED = 42
train = train.sample(frac=1, random_state = SEED)
train_index = np.arange(len(train))
print(train)
print(train_index)

In [ ]:
IMAGE_SIZE = 128

def read_image(filepath):
  return cv2.imread(os.path.join(data_dir,filepath))

def resize_image(image, image_size):
  return cv2.resize(image.copy(), image_size, interpolation=cv2.INTER_AREA)

In [ ]:
X_train = np.zeros((train.shape[0], IMAGE_SIZE, IMAGE_SIZE,3))
for i, file in tqdm(enumerate(train['File'].values)):
  image = read_image(file)
  if image is not None:
    X_train[i] = resize_image(image, (IMAGE_SIZE, IMAGE_SIZE))

X_train = X_train / 255.
print(X_train.shape)

200it [00:58,  3.44it/s]

(200, 128, 128, 3)


In [ ]:
type(X_train)

In [ ]:
num_classes = 2
y_train = []
for title in train['Disease Type']:
  if title == 'COVID':
    y_train.append(1)
  else:
    y_train.append(0)

y_train = np.array(y_train)
#pd.DataFrame(y_train)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, train_size=None, random_state=42, shuffle=False, stratify=None)
x_train

In [ ]:
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
y_train.shape

(160, 2)

In [ ]:
y_train

In [ ]:
input_shape = (IMAGE_SIZE,IMAGE_SIZE,3)
batch_size = 16
num_classes = 2
epochs = 10

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss = keras.losses.categorical_crossentropy,
             optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.01),
             metrics = ['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

Epoch 1/10
10/10 [==============================] - 19s 2s/step - loss: 25.7016 - accuracy: 0.4875 - val_loss: 0.7754 - val_accuracy: 0.4500
Epoch 2/10
10/10 [==============================] - 18s 2s/step - loss: 0.7230 - accuracy: 0.4437 - val_loss: 0.6920 - val_accuracy: 0.5500
Epoch 3/10
10/10 [==============================] - 17s 2s/step - loss: 0.6937 - accuracy: 0.4875 - val_loss: 0.6927 - val_accuracy: 0.5500
Epoch 4/10
10/10 [==============================] - 16s 2s/step - loss: 0.6939 - accuracy: 0.4875 - val_loss: 0.6923 - val_accuracy: 0.5500
Epoch 5/10
10/10 [==============================] - 17s 2s/step - loss: 0.6930 - accuracy: 0.4875 - val_loss: 0.6929 - val_accuracy: 0.5500
Epoch 6/10
10/10 [==============================] - 25s 2s/step - loss: 0.6928 - accuracy: 0.5312 - val_loss: 0.6950 - val_accuracy: 0.4500
Epoch 7/10
10/10 [==============================] - 19s 2s/step - loss: 0.6930 - accuracy: 0.5125 - val_loss: 0.6964 - val_accuracy: 0.4500
Epoch 8/10
10/10 [=